<a href="https://colab.research.google.com/github/danshal/DL_project/blob/develop/wav2vec_SV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Mount Google Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


##Install fairseq and more packages

In [2]:
!pip install soundfile git+git://github.com/pytorch/fairseq.git@b8ea8a9b72c82192da07e3377adf4ebbde16716d
!pip install fairseq
!pip install torchaudio
#!mkdir fairseq_code
#%cd /content/drive/MyDrive/DeepProject/wav2vec/fairseq_code/
#!git clone https://github.com/pytorch/fairseq.git

  Cloning git://github.com/pytorch/fairseq.git (to revision b8ea8a9b72c82192da07e3377adf4ebbde16716d) to /tmp/pip-req-build-pgso7tzk
  Running command git clone -q git://github.com/pytorch/fairseq.git /tmp/pip-req-build-pgso7tzk
  Running command git checkout -q b8ea8a9b72c82192da07e3377adf4ebbde16716d
  Running command git submodule update --init --recursive -q
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 133kB 15.2MB/s 
     |████████████████████████████████| 71kB 8.8MB/s 
     |████████████████████████████████| 112kB 32.4MB/s 
     |████████████████████████████████| 645kB 46.2MB/s 
  Created wheel for fairseq: filename=fairseq-1.0.0a0+b8ea8a9-cp36-cp36m-linux_x86_64.whl size=2713490 sha256=ede2c4c05afaaac695e3274ecf18f1ef48db3457ed399db34876beb84bed643c
  Stored in directory: /tmp/pip-ephem-wheel-cache-oimj1spi/wheels/39/

##Imports

In [1]:
#General imports
import argparse
import glob
import os
import os.path as osp
import pprint
import soundfile as sf
#facebook team framework
import fairseq
#pytorch
import torch
from torch import nn
from torch import optim
from torch.utils.data import DataLoader
import torchaudio
#matplotlib
import matplotlib.pyplot as plt
try:
    import tqdm
except:
    print("Install tqdm to use --log-format=tqdm")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cpu


/usr/local/lib/python3.6/dist-packages/torchaudio/backend/utils.py:54: UserWarning: "sox" backend is being deprecated. The default backend will be changed to "sox_io" backend in 0.8.0 and "sox" backend will be removed in 0.9.0. Please migrate to "sox_io" backend. Please refer to https://github.com/pytorch/audio/issues/903 for the detail.
  '"sox" backend is being deprecated. '


##Try sanity code

In [2]:
import torch
import fairseq

cp_path = cp_path = '/content/drive/My Drive/DeepProject/wav2vec/wav2vec_large.pt'
model, cfg, task = fairseq.checkpoint_utils.load_model_ensemble_and_task([cp_path])
model = model[0]
model.eval()

wav_input_16khz = torch.randn(1,10000)
z = model.feature_extractor(wav_input_16khz)
c = model.feature_aggregator(z)
print(f'input shape = {wav_input_16khz.shape} ; z shape = {z.shape} ; z shape = {c.shape}')

input shape = torch.Size([1, 10000]) ; z shape = torch.Size([1, 512, 60]) ; z shape = torch.Size([1, 512, 60])


In [12]:
#@title Hyperparametes
batch_size = 32
epochs = 100


###Download Dataset
###Information about LibriSpeech dataset:
1. Total of 982 hours of 16KHz read English speech
2. 2484 speakers in this corpus.
3. Suppose to be reasonably balances in terms of gender and per-speaker duration
4. Allowed urls for LIBRISPEECH - "dev-clean", "dev-other", "test-clean", "test-other", "train-clean-100", "train-clean-360" and "train-other-500". (default: "train-clean-100")

In [14]:
rootDir = '/content/drive/MyDrive/DeepProject/wav2vec/'
# Download and load the training data
train_data = torchaudio.datasets.LIBRISPEECH(f'{rootDir}/dataset/', download=True)
#train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True)
# Download and load the test data
test_data = torchaudio.datasets.LIBRISPEECH(f'{rootDir}/dataset/', url='test-clean', download=True)
#test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, shuffle=True)

In [58]:
import os
from typing import Tuple

import torchaudio
from torch import Tensor
from torch.utils.data import Dataset
from torchaudio.datasets.utils import (
    download_url,
    extract_archive,
    walk_files,
)

URL = "train-clean-100"
FOLDER_IN_ARCHIVE = "LibriSpeech"
_CHECKSUMS = {
    "http://www.openslr.org/resources/12/dev-clean.tar.gz":
    "76f87d090650617fca0cac8f88b9416e0ebf80350acb97b343a85fa903728ab3",
    "http://www.openslr.org/resources/12/dev-other.tar.gz":
    "12661c48e8c3fe1de2c1caa4c3e135193bfb1811584f11f569dd12645aa84365",
    "http://www.openslr.org/resources/12/test-clean.tar.gz":
    "39fde525e59672dc6d1551919b1478f724438a95aa55f874b576be21967e6c23",
    "http://www.openslr.org/resources/12/test-other.tar.gz":
    "d09c181bba5cf717b3dee7d4d592af11a3ee3a09e08ae025c5506f6ebe961c29",
    "http://www.openslr.org/resources/12/train-clean-100.tar.gz":
    "d4ddd1d5a6ab303066f14971d768ee43278a5f2a0aa43dc716b0e64ecbbbf6e2",
    "http://www.openslr.org/resources/12/train-clean-360.tar.gz":
    "146a56496217e96c14334a160df97fffedd6e0a04e66b9c5af0d40be3c792ecf",
    "http://www.openslr.org/resources/12/train-other-500.tar.gz":
    "ddb22f27f96ec163645d53215559df6aa36515f26e01dd70798188350adcb6d2"
}



def load_librispeech_item(fileid: str,
                          path: str,
                          ext_audio: str) -> Tuple[Tensor, int]:
    speaker_id

    # file_text = speaker_id + "-" + chapter_id + ext_txt
    # file_text = os.path.join(path, speaker_id, chapter_id, file_text)

    fileid_audio = speaker_id + "-" + chapter_id + "-" + utterance_id
    file_audio = fileid_audio + ext_audio
    file_audio = os.path.join(path, speaker_id, chapter_id, file_audio)

    # Load audio - don't care about sample rate
    waveform, _ = torchaudio.load(file_audio)

    return (waveform, int(speaker_id))


class SV_LIBRISPEECH(torchaudio.datasets.LIBRISPEECH):
  def __init__(self, root: str, url: str = 'train-clean-100', folder_in_archive: str = 'SV_LibriSpeech', download: bool = False):
    super().__init__(root, download)


  def __getitem__(self, n: int) -> Tuple[Tensor, int]:
        """Load the n-th sample from the dataset.

        Args:
            n (int): The index of the sample to be loaded

        Returns:
            tuple: ``(waveform, speaker_id)``
        """
        fileid = self._walker[n]
        return load_librispeech_item(fileid, self._path, self._ext_audio)

  def __len__(self) -> int:
    return len(self._walker)



my_train_data = SV_LIBRISPEECH(f'{rootDir}dataset/', download=True)

TypeError: ignored

In [57]:
print(len(my_train_data))


0


In [66]:
rootDir = '/content/drive/MyDrive/DeepProject/wav2vec/'

import os
from typing import Tuple

import torchaudio
from torch import Tensor
from torch.utils.data import Dataset
from torchaudio.datasets.utils import (
    download_url,
    extract_archive,
    walk_files,
)

URL = "train-clean-100"
FOLDER_IN_ARCHIVE = "LibriSpeech"
_CHECKSUMS = {
    "http://www.openslr.org/resources/12/dev-clean.tar.gz":
    "76f87d090650617fca0cac8f88b9416e0ebf80350acb97b343a85fa903728ab3",
    "http://www.openslr.org/resources/12/dev-other.tar.gz":
    "12661c48e8c3fe1de2c1caa4c3e135193bfb1811584f11f569dd12645aa84365",
    "http://www.openslr.org/resources/12/test-clean.tar.gz":
    "39fde525e59672dc6d1551919b1478f724438a95aa55f874b576be21967e6c23",
    "http://www.openslr.org/resources/12/test-other.tar.gz":
    "d09c181bba5cf717b3dee7d4d592af11a3ee3a09e08ae025c5506f6ebe961c29",
    "http://www.openslr.org/resources/12/train-clean-100.tar.gz":
    "d4ddd1d5a6ab303066f14971d768ee43278a5f2a0aa43dc716b0e64ecbbbf6e2",
    "http://www.openslr.org/resources/12/train-clean-360.tar.gz":
    "146a56496217e96c14334a160df97fffedd6e0a04e66b9c5af0d40be3c792ecf",
    "http://www.openslr.org/resources/12/train-other-500.tar.gz":
    "ddb22f27f96ec163645d53215559df6aa36515f26e01dd70798188350adcb6d2"
}


def load_librispeech_item(fileid: str,
                          path: str,
                          ext_audio: str,
                          ext_txt: str) -> Tuple[Tensor, int]:
    speaker_id

    file_text = speaker_id + "-" + chapter_id + ext_txt
    file_text = os.path.join(path, speaker_id, chapter_id, file_text)

    fileid_audio = speaker_id + "-" + chapter_id + "-" + utterance_id
    file_audio = fileid_audio + ext_audio
    file_audio = os.path.join(path, speaker_id, chapter_id, file_audio)

    # Load audio
    waveform, sample_rate = torchaudio.load(file_audio)

    # Load text
    with open(file_text) as ft:
        for line in ft:
            fileid_text, utterance = line.strip().split(" ", 1)
            if fileid_audio == fileid_text:
                break
        else:
            # Translation not found
            raise FileNotFoundError("Translation not found for " + fileid_audio)

    return (waveform, int(speaker_id))


class LIBRISPEECH_SV(Dataset):
    """Create a Dataset for LibriSpeech.

    Args:
        root (str): Path to the directory where the dataset is found or downloaded.
        url (str, optional): The URL to download the dataset from,
            or the type of the dataset to dowload.
            Allowed type values are ``"dev-clean"``, ``"dev-other"``, ``"test-clean"``,
            ``"test-other"``, ``"train-clean-100"``, ``"train-clean-360"`` and
            ``"train-other-500"``. (default: ``"train-clean-100"``)
        folder_in_archive (str, optional):
            The top-level directory of the dataset. (default: ``"LibriSpeech"``)
        download (bool, optional):
            Whether to download the dataset if it is not found at root path. (default: ``False``).
    """

    _ext_txt = ".trans.txt"
    _ext_audio = ".flac"

    def __init__(self,
                 root: str,
                 url: str = URL,
                 folder_in_archive: str = FOLDER_IN_ARCHIVE,
                 download: bool = False) -> None:

        if url in [
            "dev-clean",
            "dev-other",
            "test-clean",
            "test-other",
            "train-clean-100",
            "train-clean-360",
            "train-other-500",
        ]:

            ext_archive = ".tar.gz"
            base_url = "http://www.openslr.org/resources/12/"

            url = os.path.join(base_url, url + ext_archive)

        print(root, url, folder_in_archive, download)
        basename = os.path.basename(url)
        archive = os.path.join(root, basename)

        basename = basename.split(".")[0]
        folder_in_archive = os.path.join(folder_in_archive, basename)

        self._path = os.path.join(root, folder_in_archive)

        if download:
            if not os.path.isdir(self._path):
                if not os.path.isfile(archive):
                    checksum = _CHECKSUMS.get(url, None)
                    download_url(url, root, hash_value=checksum)
                extract_archive(archive)

        walker = walk_files(
            self._path, suffix=self._ext_audio, prefix=False, remove_suffix=True
        )
        self._walker = list(walker)

    def __getitem__(self, n: int) -> Tuple[Tensor, int]:
        """Load the n-th sample from the dataset.

        Args:
            n (int): The index of the sample to be loaded

        Returns:
            tuple: ``(waveform, sample_rate, utterance, speaker_id, chapter_id, utterance_id)``
        """
        fileid = self._walker[n]
        return load_librispeech_item(fileid, self._path, self._ext_audio, self._ext_txt)


    def __len__(self) -> int:
        return len(self._walker)

s = f'{rootDir}/dataset/'
print(type(s))
my_train_data = SV_LIBRISPEECH(f'{rootDir}/dataset/', download=True)
print(len(my_train_data))

<class 'str'>


TypeError: ignored

##Show Training Examples

In [35]:
images = next(iter(train_data))
count = 0
for i in iter(train_data):
  count = count + 1
  print(i)
  if count == 2:
    break
print(images[0].shape)  #audio samples as tensor
print(images[3])  #represents speaker id

(tensor([[-0.0065, -0.0055, -0.0062,  ...,  0.0033,  0.0005, -0.0095]]), 16000, 'CHAPTER ONE MISSUS RACHEL LYNDE IS SURPRISED MISSUS RACHEL LYNDE LIVED JUST WHERE THE AVONLEA MAIN ROAD DIPPED DOWN INTO A LITTLE HOLLOW FRINGED WITH ALDERS AND LADIES EARDROPS AND TRAVERSED BY A BROOK', 103, 1240, 0)
(tensor([[-0.0059, -0.0045, -0.0067,  ...,  0.0007,  0.0034,  0.0047]]), 16000, "THAT HAD ITS SOURCE AWAY BACK IN THE WOODS OF THE OLD CUTHBERT PLACE IT WAS REPUTED TO BE AN INTRICATE HEADLONG BROOK IN ITS EARLIER COURSE THROUGH THOSE WOODS WITH DARK SECRETS OF POOL AND CASCADE BUT BY THE TIME IT REACHED LYNDE'S HOLLOW IT WAS A QUIET WELL CONDUCTED LITTLE STREAM", 103, 1240, 1)
<class 'torch.Tensor'>
torch.Size([1, 225360])
103


In [24]:
for i, data in enumerate(train_loader, 0):
  inputs, labels = data[0].to(device), data[1].to(device)

RuntimeError: ignored

In [ ]:
waveform, sample_rate = torchaudio.load(speaker_84_data_dir)

print("Shape of waveform: {}".format(waveform.size()))
print("Sample rate of waveform: {}".format(sample_rate))

plt.figure()
plt.plot(waveform.t().numpy())

##Helper functions

In [5]:
speaker_84_data_dir = '/content/drive/MyDrive/DeepProject/wav2vec/LibriSpeech/dev-clean/84/'

def read_audio(fname):
    ''' Load an audio file and return PCM along with the sample rate '''

    wav, sr = sf.read(fname)
    assert sr == 16e3

    return wav, 16e3


def audio2tensor(fname):
  ''' This function will convert audio samples to tensor '''
  input, _ = read_audio(fname)
  input_tensor = torch.from_numpy(input).float()
  return input_tensor.reshape(1, input_tensor.size(0))


def get_audio_repr(fname):
  ''' This function will get audio as input and will output its representation
      as wav2vec model outputs '''
  input_tensor = audio2tensor(fname)
  z = model.feature_extractor(input_tensor)
  c = model.feature_aggregator(z)
  return c


#def preprocess_data(dataset_folder):



input1_tensor = audio2tensor(f'{speaker_84_data_dir}/121123/84-121123-0003.flac')
input2_tensor = audio2tensor(f'{speaker_84_data_dir}/121123/84-121123-0004.flac')
input1_repr = get_audio_repr(f'{speaker_84_data_dir}/121123/84-121123-0003.flac')
print(f'input1_tensor shape = {input1_tensor.shape} ; input2_tensor shape = {input2_tensor.shape}')
print(f'input1 repr = {input1_repr} with shape = {input1_repr.shape}')
print(input1_tensor.shape[1] / input1_repr.shape[2])
#Get whole models output
# a = model(input2_tensor)
# print(a)
# print(a['cpc_targets'].shape)
# for i in a['cpc_targets']:
#   if i != torch.tensor(0):
#     print('OK')

OSError: ignored

###Get information about audio file include the duration time in seconds

In [6]:
print(sf.info(f'{speaker_84_data_dir}/121123/84-121123-0003.flac'))

/content/drive/MyDrive/DeepProject/wav2vec/LibriSpeech/dev-clean/84/121123/84-121123-0003.flac
samplerate: 16000 Hz
channels: 1
duration: 6.800 s
format: FLAC (Free Lossless Audio Codec) [FLAC]
subtype: Signed 16 bit PCM [PCM_16]


##On Top Model

##Declare Loss functions and some Hyperparametes

##Generic Training Code